<a href="https://colab.research.google.com/github/c3045835Newcastle/2/blob/main/Part2_Coursework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predictive Analytics, Computer Vision & AI - CSC3831
## Coursework, Part 2: Machine Learning

As this coursework is as much about practical skills as it is about reflecting on the procedures and the results, you are expected to explain what you did, your reasoning for process decisions, as well as a thorough analysis of your results.

### 1. Load the MNIST dataset, visualise the first 20 digits, and print their corresponding labels.

In [ ]:
# Run this to load MNIST

import keras
import numpy as np

(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()
X = np.concatenate((X_train, X_test))
y = np.concatenate((y_train, y_test))

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
(70000, 28, 28)
(70000,)


In [ ]:
# Task 1: Visualize first 20 digits and their labels
import matplotlib.pyplot as plt

# Create a figure with 4x5 subplots to display 20 digits
fig, axes = plt.subplots(4, 5, figsize=(10, 8),
                         subplot_kw={'xticks':[], 'yticks':[]},
                         gridspec_kw={'hspace':0.3, 'wspace':0.1})

# Plot the first 20 digits
for i, ax in enumerate(axes.flat):
    ax.imshow(X[i], cmap='gray')
    ax.set_title(f'Label: {y[i]}')

plt.suptitle('First 20 MNIST Digits', fontsize=16)
plt.tight_layout()
plt.show()

# Print the corresponding labels
print("Labels for first 20 digits:")
print(y[:20])

### 2. Train a Logistic Regression classifier on this data, and report on your findings.
    
1. Tune your hyperparameters to ensure *sparse* weight vectors and high accuracy.
2. Visualise the classification vector for each class.

In [ ]:
# Task 2: Train a Logistic Regression classifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# Flatten the images from 28x28 to 784-dimensional vectors
X_flat = X.reshape(X.shape[0], -1)

# Normalize pixel values to [0, 1]
X_flat = X_flat / 255.0

# Split the data into training and test sets
X_train_lr, X_test_lr, y_train_lr, y_test_lr = train_test_split(
    X_flat, y, test_size=0.2, random_state=42
)

print("Training Logistic Regression with hyperparameter tuning...")
print("This may take several minutes.")

# Use L1 penalty for sparse weight vectors
# GridSearchCV to find optimal C parameter
param_grid = {
    'C': [0.01, 0.1, 1, 10],
}

lr = LogisticRegression(
    penalty='l1',
    solver='saga',
    max_iter=100,
    random_state=42,
    n_jobs=-1
)

grid_search = GridSearchCV(
    lr, param_grid, cv=3, scoring='accuracy', n_jobs=-1, verbose=1
)

grid_search.fit(X_train_lr, y_train_lr)

print(f"\nBest parameters: {grid_search.best_params_}")
print(f"Best cross-validation accuracy: {grid_search.best_score_:.4f}")

# Get the best model
best_lr = grid_search.best_estimator_

# Evaluate on test set
y_pred = best_lr.predict(X_test_lr)
test_accuracy = accuracy_score(y_test_lr, y_pred)
print(f"Test accuracy: {test_accuracy:.4f}")

# Print classification report
print("\nClassification Report:")
print(classification_report(y_test_lr, y_pred))

# Check sparsity of weight vectors
coef = best_lr.coef_
sparsity = np.mean(coef == 0)
print(f"\nSparsity of weight vectors: {sparsity:.2%}")
print(f"Non-zero weights per class (avg): {np.mean(np.sum(coef != 0, axis=1)):.0f}")

In [ ]:
# Visualize the classification vectors for each class
fig, axes = plt.subplots(2, 5, figsize=(12, 6),
                         subplot_kw={'xticks':[], 'yticks':[]},
                         gridspec_kw={'hspace':0.3, 'wspace':0.1})

for i, ax in enumerate(axes.flat):
    # Reshape the coefficient vector back to 28x28 image
    weight_image = best_lr.coef_[i].reshape(28, 28)
    ax.imshow(weight_image, cmap='RdBu', vmin=-weight_image.max(), vmax=weight_image.max())
    ax.set_title(f'Class {i}')

plt.suptitle('Classification Weight Vectors for Each Digit Class', fontsize=16)
plt.tight_layout()
plt.show()

print("\nInterpretation:")
print("Red regions indicate positive weights (features that support this class).")
print("Blue regions indicate negative weights (features that oppose this class).")
print("White/gray regions are near-zero weights (sparse - not important for classification).")

### 3. Use PCA to reduce the dimensionality of your training data.
    
1. Determine the number of components necessary to explain 80\% of the variance
2. Plot the explained variance by number of components.
3. Visualise the 20 principal components' loadings
4. Plot the two principal components for your data using a scatterplot, colouring by class. What can you say about this plot?
5. Visualise the first 20 digits, *generated from their lower-dimensional representation*.

In [ ]:
# Task 3: Use PCA to reduce dimensionality
from sklearn.decomposition import PCA

# Use the flattened and normalized data
print("Applying PCA to determine components for 80% variance...")

# First, fit PCA with all components to analyze variance
pca_full = PCA()
pca_full.fit(X_flat)

# Calculate cumulative explained variance
cumsum_variance = np.cumsum(pca_full.explained_variance_ratio_)

# Find number of components for 80% variance
n_components_80 = np.argmax(cumsum_variance >= 0.80) + 1
print(f"\nNumber of components needed for 80% variance: {n_components_80}")
print(f"Exact variance explained with {n_components_80} components: {cumsum_variance[n_components_80-1]:.4f}")

In [ ]:
# Plot explained variance by number of components
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(cumsum_variance) + 1), cumsum_variance, linewidth=2)
plt.axhline(y=0.80, color='r', linestyle='--', label='80% Variance')
plt.axvline(x=n_components_80, color='g', linestyle='--', 
            label=f'{n_components_80} components')
plt.xlabel('Number of Components', fontsize=12)
plt.ylabel('Cumulative Explained Variance', fontsize=12)
plt.title('PCA Explained Variance vs Number of Components', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.xlim(0, 300)
plt.show()

# Also show individual variance for first 50 components
plt.figure(figsize=(10, 6))
plt.bar(range(1, 51), pca_full.explained_variance_ratio_[:50])
plt.xlabel('Component Number', fontsize=12)
plt.ylabel('Explained Variance Ratio', fontsize=12)
plt.title('Individual Explained Variance for First 50 Components', fontsize=14)
plt.grid(True, alpha=0.3, axis='y')
plt.show()

In [ ]:
# Visualize the first 20 principal components' loadings
fig, axes = plt.subplots(4, 5, figsize=(12, 10),
                         subplot_kw={'xticks':[], 'yticks':[]},
                         gridspec_kw={'hspace':0.3, 'wspace':0.1})

for i, ax in enumerate(axes.flat):
    # Reshape the component back to 28x28
    component_image = pca_full.components_[i].reshape(28, 28)
    ax.imshow(component_image, cmap='viridis')
    ax.set_title(f'PC {i+1}\n({pca_full.explained_variance_ratio_[i]:.3f})', 
                 fontsize=10)

plt.suptitle('First 20 Principal Component Loadings', fontsize=16)
plt.tight_layout()
plt.show()

print("\nInterpretation:")
print("Each principal component represents a pattern of variation in the data.")
print("The percentage shows how much variance each component explains.")
print("Earlier components capture broader patterns, later ones capture finer details.")

In [ ]:
# Plot the first two principal components as a scatter plot
print("Transforming data to 2D using first two principal components...")
pca_2d = PCA(n_components=2)
X_2d = pca_2d.fit_transform(X_flat)

# Create scatter plot colored by class
plt.figure(figsize=(12, 10))

# Use a colormap with 10 distinct colors
colors = plt.cm.tab10(np.linspace(0, 1, 10))

for digit in range(10):
    mask = y == digit
    plt.scatter(X_2d[mask, 0], X_2d[mask, 1], 
                c=[colors[digit]], label=str(digit), 
                alpha=0.6, s=10)

plt.xlabel(f'First Principal Component ({pca_2d.explained_variance_ratio_[0]:.3f})', 
           fontsize=12)
plt.ylabel(f'Second Principal Component ({pca_2d.explained_variance_ratio_[1]:.3f})', 
           fontsize=12)
plt.title('MNIST Digits Projected onto First Two Principal Components', fontsize=14)
plt.legend(title='Digit', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\nAnalysis of the 2D PCA plot:")
print("- The plot shows how different digit classes are distributed in the space")
print("  defined by the two most important principal components.")
print("- Some digit classes form distinct clusters (e.g., 0s and 1s are often separated).")
print("- Other classes show significant overlap, indicating similar patterns in these")
print("  two principal dimensions (e.g., 4s, 7s, and 9s may overlap).")
print("- The first two components only explain a small fraction of total variance,")
print("  so this is a very compressed representation of the data.")
print("- The plot demonstrates that even with just 2 dimensions, some structure")
print("  of the digit classes is preserved, but full separation requires more dimensions.")

In [ ]:
# Visualize first 20 digits generated from lower-dimensional representation
print("Reconstructing digits from lower-dimensional representation...")

# Use PCA with components for 80% variance
pca_reduced = PCA(n_components=n_components_80)
X_reduced = pca_reduced.fit_transform(X_flat)

# Reconstruct the images from the reduced representation
X_reconstructed = pca_reduced.inverse_transform(X_reduced)

# Visualize original vs reconstructed
fig, axes = plt.subplots(4, 10, figsize=(15, 6),
                         subplot_kw={'xticks':[], 'yticks':[]},
                         gridspec_kw={'hspace':0.3, 'wspace':0.1})

for i in range(20):
    # Original image (top two rows)
    axes[i // 10 * 2, i % 10].imshow(X[i], cmap='gray')
    if i < 10:
        axes[0, i].set_title(f'Original\n{y[i]}', fontsize=9)
    
    # Reconstructed image (bottom two rows)
    reconstructed_img = X_reconstructed[i].reshape(28, 28)
    axes[i // 10 * 2 + 1, i % 10].imshow(reconstructed_img, cmap='gray')
    if i < 10:
        axes[1, i].set_title('Reconstructed', fontsize=9)

plt.suptitle(f'Original vs Reconstructed Digits (using {n_components_80} components, 80% variance)', 
             fontsize=14)
plt.tight_layout()
plt.show()

print(f"\nDigits were compressed from 784 dimensions to {n_components_80} dimensions.")
print("The reconstructed images show good preservation of the essential digit features.")
print("Some fine details are lost, but the digits remain recognizable.")

### 4. Generate a noisy copy of your data by adding random normal noise to the digits **with a scale that doesn't completely destroy the signal**. This is, the resulting images noise should be apparent, but the numbers should still be understandable.
    
1. Visualise the first 20 digits from the noisy dataset.
2. Filter the noise by fitting a PCA explaining **a sufficient proportion** of the variance, and then transforming the noisy dataset. Figuring out this proportion is part of the challenge.
3. Visualise the first 20 digits of the de-noised dataset.

In [ ]:
# Task 4: Generate noisy data and denoise with PCA
print("Generating noisy copy of MNIST data...")

# Add random normal noise with appropriate scale
# Scale of 0.3 is chosen to be noticeable but not overwhelming
noise_scale = 0.3
noise = np.random.normal(0, noise_scale, X_flat.shape)
X_noisy = X_flat + noise

# Clip values to valid range [0, 1]
X_noisy = np.clip(X_noisy, 0, 1)

print(f"Added Gaussian noise with mean=0 and std={noise_scale}")
print(f"Noise-to-signal ratio: {noise_scale:.2f}")

In [ ]:
# Visualize the first 20 digits from the noisy dataset
fig, axes = plt.subplots(4, 5, figsize=(10, 8),
                         subplot_kw={'xticks':[], 'yticks':[]},
                         gridspec_kw={'hspace':0.3, 'wspace':0.1})

for i, ax in enumerate(axes.flat):
    noisy_img = X_noisy[i].reshape(28, 28)
    ax.imshow(noisy_img, cmap='gray', vmin=0, vmax=1)
    ax.set_title(f'Label: {y[i]}')

plt.suptitle(f'First 20 Noisy MNIST Digits (noise std={noise_scale})', fontsize=16)
plt.tight_layout()
plt.show()

print("\nObservation: The noise is visible but the digits are still recognizable.")

In [ ]:
# Denoise using PCA
print("Denoising using PCA...")

# For denoising, we want to capture the main signal while filtering out noise
# We'll use slightly more components than for 80% variance to preserve details
# Typically, 90-95% variance works well for denoising
variance_threshold = 0.90

# Fit PCA on the original (clean) data to learn the true signal structure
pca_denoise = PCA(n_components=variance_threshold, svd_solver='full')
pca_denoise.fit(X_flat)

n_components_denoise = pca_denoise.n_components_
print(f"\nUsing {n_components_denoise} components ({variance_threshold*100}% variance)")
print(f"This filters out the remaining {(1-variance_threshold)*100}% variance,")
print("which primarily consists of noise and fine details.")

# Transform noisy data and reconstruct (denoise)
X_noisy_transformed = pca_denoise.transform(X_noisy)
X_denoised = pca_denoise.inverse_transform(X_noisy_transformed)

# Ensure values are in valid range
X_denoised = np.clip(X_denoised, 0, 1)

print("\nDenoising complete!")

In [ ]:
# Visualize the first 20 digits of the denoised dataset
fig, axes = plt.subplots(4, 5, figsize=(10, 8),
                         subplot_kw={'xticks':[], 'yticks':[]},
                         gridspec_kw={'hspace':0.3, 'wspace':0.1})

for i, ax in enumerate(axes.flat):
    denoised_img = X_denoised[i].reshape(28, 28)
    ax.imshow(denoised_img, cmap='gray', vmin=0, vmax=1)
    ax.set_title(f'Label: {y[i]}')

plt.suptitle(f'First 20 Denoised MNIST Digits ({n_components_denoise} components)', 
             fontsize=16)
plt.tight_layout()
plt.show()

print("\nObservation: The denoised images show significant noise reduction")
print("while preserving the essential digit features.")

In [ ]:
# Compare original, noisy, and denoised side by side
fig, axes = plt.subplots(3, 10, figsize=(15, 5),
                         subplot_kw={'xticks':[], 'yticks':[]},
                         gridspec_kw={'hspace':0.3, 'wspace':0.1})

for i in range(10):
    # Original
    axes[0, i].imshow(X[i], cmap='gray')
    axes[0, i].set_title(f'{y[i]}', fontsize=10)
    
    # Noisy
    axes[1, i].imshow(X_noisy[i].reshape(28, 28), cmap='gray', vmin=0, vmax=1)
    
    # Denoised
    axes[2, i].imshow(X_denoised[i].reshape(28, 28), cmap='gray', vmin=0, vmax=1)

# Add row labels
axes[0, 0].set_ylabel('Original', fontsize=12, rotation=0, ha='right', va='center')
axes[1, 0].set_ylabel('Noisy', fontsize=12, rotation=0, ha='right', va='center')
axes[2, 0].set_ylabel('Denoised', fontsize=12, rotation=0, ha='right', va='center')

plt.suptitle('Comparison: Original vs Noisy vs Denoised Digits', fontsize=16)
plt.tight_layout()
plt.show()

# Calculate reconstruction quality
mse_noisy = np.mean((X_flat[:20] - X_noisy[:20])**2)
mse_denoised = np.mean((X_flat[:20] - X_denoised[:20])**2)

print(f"\nMean Squared Error (first 20 digits):")
print(f"  Noisy vs Original: {mse_noisy:.6f}")
print(f"  Denoised vs Original: {mse_denoised:.6f}")
print(f"\nNoise reduction: {(1 - mse_denoised/mse_noisy)*100:.1f}%")
print("\nConclusion:")
print(f"PCA successfully reduced noise by projecting the data onto {n_components_denoise}")
print("principal components that capture the main signal structure, while filtering")
print("out the noise components. The denoised images are much closer to the originals.")